In [1]:
import numpy as np
import pandas as pd
import os
import glob
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive
from google.colab import files

In [2]:
#dat=os.path.join(file,"Desktop","data")
#drive.mount('/conect/drive')
drive.mount('/content/drive')
!ls /content/drive/MyDrive/data
!cp -r /content/drive/MyDrive/data /content/local_data

#dat=files.upload()
#data=pd.read_csv(data)
activity_labels=['idle', 'running', "stairs", "walking"]
files={label:[] for label in activity_labels}

Mounted at /content/drive
idle  running  stairs  walking


In [3]:
data='/content/local_data'



In [4]:
for root,dirs,filles in os.walk(data):
    print(root)

    for file in filles:
        for label in activity_labels :
            if label in file and  file.endswith("csv"):
                path = os.path.join(root, file)
                try:
                    read = pd.read_csv(path)
                    read['activity'] = label
                    files[label].append(read)
                    print(f"{path}")
                except Exception as e:
                    print(f"Błąd podczas wczytywania pliku: {path}")
                    print(f"Szczegóły błędu: {e}")


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
/content/local_data/walking/walking-1218.csv
/content/local_data/walking/walking-1733.csv
/content/local_data/walking/walking-1804.csv
/content/local_data/walking/walking-428.csv
/content/local_data/walking/walking-1271.csv
/content/local_data/walking/walking-1213.csv
/content/local_data/walking/walking-790.csv
/content/local_data/walking/walking-318.csv
/content/local_data/walking/walking-1416.csv
/content/local_data/walking/walking-128.csv
/content/local_data/walking/walking-1000.csv
/content/local_data/walking/walking-1677.csv
/content/local_data/walking/walking-551.csv
/content/local_data/walking/walking-1595.csv
/content/local_data/walking/walking-69.csv
/content/local_data/walking/walking-393.csv
/content/local_data/walking/walking-230.csv
/content/local_data/walking/walking-260.csv
/content/local_data/walking/walking-1298.csv
/content/local_data/walking/walking-1727.csv
/content/local_data/walking/walking-1427.csv


In [5]:
if any(files.values()):
    data_fatms=[]
    for label in activity_labels:
        if files[label]:
            data_fatms.append(pd.concat(files[label], ignore_index=True))
    data=pd.concat(data_fatms, ignore_index=True)
    data=data.dropna()
    x = data[["accelerometer_X", "accelerometer_Y", "accelerometer_Z"]].values
    y = data["activity"].values
    scalar = StandardScaler()
    data_scaled = scalar.fit_transform(x)
    X_train, X_test, y_train, y_test = train_test_split(data_scaled,y, test_size=0.4, random_state=40)

model_rf =  RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(X_train, y_train)
model_svc = SVC(kernel="rbf", C=1, gamma="auto")
model_svc.fit(X_train, y_train)
model_rf =  RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(X_train, y_train)
model_svc = SVC(kernel="rbf", C=1, gamma="auto")
model_svc.fit(X_train, y_train)

SVC(C=1, gamma='auto')

In [6]:
predic_svm=model_svc.predict(X_test)
perfect_svm=accuracy_score(y_test, predic_svm)
predic_rf=model_rf.predict(X_test)
perfect_rf=accuracy_score(y_test, predic_rf)

In [34]:
data['accelerometer_X_mean']= data['accelerometer_X'].rolling(window=10).mean()
data['accelerometer_X_std']= data['accelerometer_X'].rolling(window=10).std()
data['accelerometer_X_min']= data['accelerometer_X'].rolling(window=10).min()
data['accelerometer_X_max']= data['accelerometer_X'].rolling(window=10).max()
data['accelerometer_X_median']= data['accelerometer_X'].rolling(window=10).median()

data['accelerometer_Y_mean']= data['accelerometer_Y'].rolling(window=10).mean()
data['accelerometer_Y_std']= data['accelerometer_Y'].rolling(window=10).std()
data['accelerometer_Y_min']=data['accelerometer_Y'].rolling(window=10).min()
data['accelerometer_Y_max']=data['accelerometer_Y'].rolling(window=10).max()
data['accelerometer_Y_median']= data['accelerometer_Y'].rolling(window=10).median()

data['accelerometer_Z_mean']= data['accelerometer_Z'].rolling(window=10).mean()
data['accelerometer_Z_std']= data['accelerometer_Z'].rolling(window=10).std()
data['accelerometer_Z_min']= data['accelerometer_Z'].rolling(window=10).min()
data['accelerometer_Z_max']= data['accelerometer_Z'].rolling(window=10).max()
data['accelerometer_Z_median']= data['accelerometer_Z'].rolling(window=10).median()

data= data.dropna()

fernitur=['accelerometer_X_mean','accelerometer_X_std','accelerometer_X_min','accelerometer_X_max','accelerometer_X_median',
    'accelerometer_Y_mean','accelerometer_Y_std','accelerometer_Y_min','accelerometer_Y_max','accelerometer_Y_median',
    'accelerometer_Z_mean','accelerometer_Z_std','accelerometer_Z_min','accelerometer_Z_max','accelerometer_Y_median']
x=data[fernitur].values

data_scalar=scalar.fit_transform(x)
if len(data_scalar)!= len(y):
    o=(len(y))
    p=(len(data_scalar))


    q=p-o
    if  o>p:
      y=np.delete(y,range(-q))
    if p>o:
      data_scalar=np.delete(data_scalar,range(-q))
else:
  y=y
  data_scalar=data_scalar
print(f"X ma {len(data_scalar)}")
print(f"y ma {len(y)}")
#print(p)
#q=p-o
#print((q))
#y=np.delete(y,range(0,q))
#print(len(y))

X ma 193698
y ma 193698


In [29]:
x_train, x_test, y_train, y_test = train_test_split(data_scalar, y, test_size=0.4, random_state=40)
model={'SVM': model_svc,"Losowy":model_rf}
score={}

for names_model, modeles in model.items():
        score[names_model]=cross_val_score(modeles,x_train,y_train,cv=5).mean()
        score[names_model]=score

for names_model, scor in score.items():
        print(f'{names_model}:{score}')

SVM:{'SVM': {...}, 'Losowy': {...}}
Losowy:{'SVM': {...}, 'Losowy': {...}}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')